# Oxytocin analysis code

In [47]:
import utility_functions # loading and processing code

# matplotlib stuff
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon # errorbars and standard deviations
%matplotlib qt

# data analysis standards
import numpy as np
import pandas as pd
from scipy import signal
import os, re
from pathlib import Path

# csv writing for figure data etc
import csv
import xmltodict

In [45]:
!pip install xmltodict

# Data import and cleaning

Bring in the data, do some basic cleaning before sending it to Kilosort etc

In [2]:
# recording directory
# assume all subdirectories are from the same session and to be concatenated
# recording_dir = Path('Y:\\Sara\\in vivo phys\\open_ephys\\Mudd data\\awake\\oxt-cre_male_2505\\oxt-cre_m2505_2025-03-17_14-25-13')
recording_dir = 'Y:\\Sara\\in vivo phys\\open_ephys\\Mudd data\\awake\\oxt-cre_male_2506\\oxt-cre_m2506_2025-03-20_14-18-47'

sig = utility_functions.open_raw(recording_dir)


NameError: name 'sig' is not defined

In [3]:
from open_ephys.analysis import Session

session = Session(recording_dir)


In [17]:
session.recordnodes[0].recordings[0].events.timestamp.describe()

count    4.320000e+04
mean     1.288912e+08
std      4.121014e+07
min      6.269484e+07
25%      9.477002e+07
50%      1.308649e+08
75%      1.640722e+08
max      1.901784e+08
Name: timestamp, dtype: float64

In [ ]:
# dir(session.recordnodes[0].recordings[0])
# dir(session)
# dir(session.recordnodes[0].recordings[0].continuous[0])
session.recordnodes[0].recordings[0].continuous[0].metadata

In [65]:
# from glob import glob


settings_file = glob(f'{recording_dir}{path.sep}**{path.sep}*settings.xml',recursive=True)

if len(settings_file)!=1:
    print('Cannot find a single settings file')


with open(settings_file[0],'r', encoding='utf8') as fid:
#     # settings = fid.readlines()
    settings = xmltodict.parse(fid.read())['SETTINGS']
# settings = xmltodict.parse(settings_file[0])

In [67]:
settings.

dict_keys(['INFO', 'SIGNALCHAIN', 'AUDIO', 'GLOBAL_TIMESTAMP', 'CONTROLPANEL', 'AUDIOEDITOR', 'PROCESSORLIST', 'UICOMPONENT'])

In [74]:
pd.DataFrame(settings['SIGNALCHAIN'])

,PROCESSOR
0,"{'@name': 'Sources/Rhythm FPGA', '@insertionPo..."
1,"{'@name': 'Filters/Record Node', '@insertionPo..."
2,"{'@name': 'Sinks/LFP Viewer', '@insertionPoint..."
3,"{'@name': 'Sinks/Pulse Pal', '@insertionPoint'..."
